In [36]:

url = ""
with open("../app.env") as file:
    db_source = file.read()
    url = db_source.split("DB_SOURCE=\"")[1][:-1]

In [37]:
from pymongo import MongoClient
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import time
client = MongoClient(url)
db = client.nba

In [38]:
from nba_api.stats.static import teams

def lookupTeam(abbreviation):
    original = abbreviation
    if abbreviation == "SEA":
        abbreviation = "OKC"
    if abbreviation == "NOH" or abbreviation == "NOK" or abbreviation == "NOR":
        abbreviation = "NOP"
    if abbreviation == "NJN":
        abbreviation = "BKN"
    ret = [team for team in teams.get_teams() if team['abbreviation'] == abbreviation]
    if len(ret) == 0:
        print(original)
    return ret[0]


In [39]:
from datetime import datetime
lastModifiedDate = datetime.now()
print(lastModifiedDate)

In [ ]:
# # get teams

# for team in teams.get_teams():
#     team_data = {
#         "teamID": team['id'],
#         'name': team['nickname'],
#         'abbreviation': team['abbreviation'],
#         "city": team["city"],
#     }
#     if team["abbreviation"] == "OKC":
#         team_data["alternate_abbreviations"] = ["SEA"]
#     if team["abbreviation"] == "NOP":
#         team_data["alternate_abbreviations"] = ["NOK", "NOH"]
#     if team["abbreviation"] == "BKN":
#         team_data["alternate_abbreviations"] = ["NJN"]
#     db.teams.update_many({"teamID": team['id']}, {"$set": team_data}, True)

In [40]:
# add players to db
#TODO: query all players from db at once instead of one at a time
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.library.parameters import PerModeSimple

# all_players = commonallplayers.CommonAllPlayers()
# data = all_players.get_normalized_dict()
# count = 0
# for player in data['CommonAllPlayers']:
#     if player["ROSTERSTATUS"] == 1:
#         names = player["DISPLAY_FIRST_LAST"].split(" ", 1)
#         player_data = {
#             "playerID":player["PERSON_ID"],
#             "first_name":names[0],
#             "last_name":names[1],
#             "teamABR": player["TEAM_ABBREVIATION"],
#             "seasons":[],
#         }
#         player_career = playercareerstats.PlayerCareerStats(player["PERSON_ID"], per_mode36=PerModeSimple.per_game)
#         regular_seasons = player_career.get_normalized_dict()["SeasonTotalsRegularSeason"]
#         time.sleep(.8)
#         for season in regular_seasons:
#             if season["MIN"] > 8.0:
#                 player_data["seasons"].append(season["SEASON_ID"])
#         if len(player_data["seasons"]) > 0:
#             info = commonplayerinfo.CommonPlayerInfo(player["PERSON_ID"]).get_normalized_dict()["CommonPlayerInfo"][0]
#             position=info["POSITION"][0]
#             player_data["position"]=position
#             player_data["height"]=info["HEIGHT"]
#             player_data["weight"]=int(info["WEIGHT"])
#             player_data["lastModifiedDate"] = lastModifiedDate
#             db.players.update_one({"playerID": season["PLAYER_ID"]}, {"$set":player_data}, True)
#             count+=1
#             print(str(count) + ": " + player["DISPLAY_FIRST_LAST"] + " " + position)
    

In [41]:
def game_exists(games, game_id):
    for game in games:
        if game["gameID"] == game_id:
            if "lastModifiedDate" in game.keys():
                # last modified after quality assurance date
                if game["lastModifiedDate"] > datetime.strptime("2022-04-20T11:50:00", "%Y-%m-%dT%H:%M:%S"):
                    return True
    return False

In [43]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoremiscv2

sleeptime = .6

def get_game_data(g, count, playoffs):
    try:
        time.sleep(sleeptime)
        boxscorefinder = boxscoretraditionalv2.BoxScoreTraditionalV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        summary = boxscoresummaryv2.BoxScoreSummaryV2(g["GAME_ID"]).get_normalized_dict()["GameSummary"][0]
        time.sleep(sleeptime)
        advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(g["GAME_ID"]).get_normalized_dict()
        time.sleep(sleeptime)
        misc = boxscoremiscv2.BoxScoreMiscV2(g["GAME_ID"]).get_normalized_dict()
        season = summary["SEASON"] + "-"+ str(int(summary["SEASON"][2:])+1)
        #team stats
        # print(count, " ", "VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
        for i in range(2):
            game = boxscorefinder["TeamStats"][i]
            if i == 1:
                opponentgame = boxscorefinder["TeamStats"][0]
            else:
                opponentgame = boxscorefinder["TeamStats"][1]
            #if the team's game stats haven't been added already
            if game["PTS"] != None:
                home_or_away = ""
                if game["TEAM_ID"] == summary["HOME_TEAM_ID"]:
                    home_or_away = "home"
                elif game["TEAM_ID"] == summary["VISITOR_TEAM_ID"]:
                    home_or_away = "away"
                else:
                    print("Error: team id not found")
                    exit(0)
                opponent = opponentgame["TEAM_ID"]
                win_or_loss = "loss"
                if game["PTS"] > opponentgame["PTS"]:
                    win_or_loss = "win"
                team_game_data = {
                    "date": summary["GAME_DATE_EST"],
                    "margin": game["PTS"] - opponentgame["PTS"],
                    "points": game["PTS"],
                    "assists": game["AST"],
                    "rebounds": game["REB"],
                    "offensive_rebounds": game["OREB"],
                    "defensive_rebounds": game["DREB"],
                    "turnovers": game["TO"],
                    "steals": game["STL"],
                    "blocks": game["BLK"],
                    "three_pointers_attempted": game["FG3A"],
                    "three_pointers_made": game["FG3M"],
                    "defensive_rating": advanced["TeamStats"][i]["DEF_RATING"],
                    "defensive_rebound_percentage": advanced["TeamStats"][i]["DREB_PCT"],
                    "field_goal_percentage": game["FG_PCT"],
                    "field_goals_attempted": game["FGA"],
                    "field_goals_made": game["FGM"],
                    "gameID":game["GAME_ID"],
                    "home_or_away":home_or_away,
                    "offfensive_rebound_percentage": advanced["TeamStats"][i]["OREB_PCT"],
                    "opponent":opponent,
                    "opponent_assists": opponentgame["AST"],
                    "opponent_effective_field_goal_percentage": opponentgame["FG_PCT"],
                    "opponent_field_goals_attempted": opponentgame["FGA"],
                    "opponent_free_throws_attempted": opponentgame["FTA"],
                    "opponent_points": opponentgame["PTS"],
                    "opponent_rebounds": opponentgame["REB"],
                    "opponent_three_pointers_attempted": opponentgame["FG3A"],
                    "opponent_three_pointers_made": opponentgame["FG3M"],
                    "personal_fouls": misc["sqlTeamsMisc"][i]["PF"],
                    "personal_fouls_drawn": misc["sqlTeamsMisc"][i]["PFD"],
                    "plus_minus_per_hundred": ((int(game["PTS"])-int(opponentgame["PTS"]))/float(advanced["TeamStats"][i]["POSS"]))*100,
                    "possessions": advanced["TeamStats"][i]["POSS"],
                    "season": season,
                    "teamID": game["TEAM_ID"],
                    "win_or_loss":win_or_loss,
                    "playoffs": playoffs,
                    "lastModifiedDate":lastModifiedDate
                }
                # print(db.teamgames.find_one({"teamID": team_game_data["teamID"], "gameID":team_game_data["gameID"]}))
                db.teamgames.replace_one({"teamID": team_game_data["teamID"], "gameID":team_game_data["gameID"]}, team_game_data, True)
                # print("T: ", game["TEAM_NAME"], " vs ", opponentgame["TEAM_NAME"], team_game_data["season"], team_game_data["gameID"])
                print("\r ",count, game["TEAM_NAME"], " vs ", opponentgame["TEAM_NAME"], team_game_data["season"], team_game_data["gameID"], end="")
        #players stats
        for game in boxscorefinder["PlayerStats"]:
            #if the player played in te game, and the player exists with the provided season (averaged > 10 min in the season), and this player's game stats haven't been added already
            if game["MIN"] != None and db.players.count_documents({"playerID": game["PLAYER_ID"], "seasons":season}) > 0:
                # print(game)
                team1 = lookupTeam(g["MATCHUP"].split(" ")[0])['id']
                team2 = lookupTeam(g["MATCHUP"].split(" ")[2])['id']
                home_or_away = ""
                team = ""
                opponent = ""
                if team1 == game["TEAM_ID"]:
                    team = team1
                    opponent = team2
                    if g["MATCHUP"].split(" ")[1] == "@":
                        #team1 is away
                        home_or_away = "away"
                    else:
                        #team1 is home
                        home_or_away = "home"
                elif team2 == game["TEAM_ID"]:
                    team = team2
                    opponent = team1
                    if g["MATCHUP"].split(" ")[1] == "@":
                        #team1 is away
                        home_or_away = "home"
                    else:
                        #team1 is home
                        home_or_away = "away"
                else:
                    print("ERROR: Team not found")
                margin = 0
                if boxscorefinder["TeamStats"][0]["TEAM_ID"] == team:
                    margin = boxscorefinder["TeamStats"][0]["PTS"] - boxscorefinder["TeamStats"][1]["PTS"]
                else:
                    margin = boxscorefinder["TeamStats"][1]["PTS"] - boxscorefinder["TeamStats"][0]["PTS"]
                win_or_loss="win"
                if margin < 0:
                    win_or_loss="loss"
                game_data = {
                    "assists":game["AST"], 
                    "date": g["GAME_DATE"],
                    "margin": margin,
                    "win_or_loss":win_or_loss,
                    "defensive_rebounds":game["DREB"], 
                    "field_goal_percentage":game["FG_PCT"], 
                    "field_goals_attempted":game["FGA"], 
                    "field_goals_made":game["FGM"], 
                    "free_throws_attempted":game["FTA"],
                    "free_throws_made":game["FTM"], 
                    "free_throws_percentage":game["FT_PCT"],
                    "gameID":game["GAME_ID"],
                    "home_or_away":home_or_away,
                    "minutes": game["MIN"],
                    "offensive_rebounds":game["OREB"], 
                    "opponent":opponent,
                    "personal_fouls":game["PF"], 
                    "player":game["PLAYER_ID"],
                    "blocks":game["BLK"],
                    "steals":game["STL"],
                    "points": game["PTS"],
                    "season": season,
                    "team": team,
                    "three_point_percentage":game["FG3_PCT"], 
                    "three_pointers_attempted":game["FG3A"], 
                    "three_pointers_made":game["FG3M"], 
                    "total_rebounds":game["REB"], 
                    "turnovers":game["TO"], 
                    "playoffs": playoffs,
                    "lastModifiedDate":lastModifiedDate
                }
                for advanced_game in advanced["PlayerStats"]:
                    if game_data['player'] == advanced_game["PLAYER_ID"]:
                        game_data["assist_percentage"] = advanced_game["AST_PCT"]
                        game_data["defensive_rebound_percentage"] = advanced_game["DREB_PCT"]
                        game_data["effective_field_goal_percentage"] = advanced_game["EFG_PCT"]
                        game_data["offensive_rebound_percentage"] = advanced_game["OREB_PCT"]
                        game_data["true_shooting_percentage"] = advanced_game["TS_PCT"]
                        game_data["usage"] = advanced_game["USG_PCT"]
                for misc_game in misc["sqlPlayersMisc"]:
                    if game_data['player'] == misc_game["PLAYER_ID"]:
                        game_data["personal_fouls_drawn"] = misc_game["PFD"]
                db.games.replace_one({"player": game_data["player"], "gameID":game_data["gameID"]}, game_data, True)
                # print(db.games.find_one({"player": game_data["player"], "gameID":game_data["gameID"]}))
                # print("\t", count, " P: ", game["PLAYER_NAME"], game_data["season"], game_data["date"], game_data["minutes"])
        # print(" "*len(str(count)), " ", "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    except Exception as e:
        print(e)
        print(g)

attempt: 0
1 From team:  Hawks
3123  total games
80  recent games
546  games found in db
1  games not in db
1   VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
T:  Wizards  vs  Hawks 2021-22 0022101192
T:  Hawks  vs  Wizards 2021-22 0022101192
	 1  P:  Kentavious Caldwell-Pope 2021-22 2022-04-06 26:00
	 1  P:  Rui Hachimura 2021-22 2022-04-06 31:41
	 1  P:  Kristaps Porzingis 2021-22 2022-04-06 32:21
	 1  P:  Corey Kispert 2021-22 2022-04-06 32:05
	 1  P:  Tomas Satoransky 2021-22 2022-04-06 25:28
	 1  P:  Deni Avdija 2021-22 2022-04-06 33:32
	 1  P:  Daniel Gafford 2021-22 2022-04-06 14:24
	 1  P:  Ish Smith 2021-22 2022-04-06 22:32
	 1  P:  Anthony Gill 2021-22 2022-04-06 13:11
	 1  P:  Raul Neto 2021-22 2022-04-06 4:23
	 1  P:  Thomas Bryant 2021-22 2022-04-06 4:23
	 1  P:  De'Andre Hunter 2021-22 2022-04-06 27:02
	 1  P:  Danilo Gallinari 2021-22 2022-04-06 39:10
	 1  P:  Clint Capela 2021-22 2022-04-06 25:09
	 1  P:  Kevin Huerter 2021-22 2022-04-06 28:33
	 1  P:  Trae Young 20

In [ ]:
#REGULAR SEASON
attempt = 0
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        teams_found = db.teams.find()
        team_count = 0
        for team in teams_found:
            team_count+=1
            print()
            print(str(team_count), "From team: ", team["name"])
            time.sleep(sleeptime)
            playoffs=False
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="00", season_type_nullable="Regular Season")
            games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            print("  ", len(games), " total games")
            games = [game for game in games if int(game["SEASON_ID"][1:]) == 2021]
            print("  ", len(games), " recent games")
            found_games = [game for game in db.teamgames.find({"teamID":team["teamID"]})]
            print("  ", len(found_games), " games found in db")
            games = [game for game in games if game_exists(found_games, game["GAME_ID"]) == False]
            print("  ", len(games), " games not in db")

            count=0
            for g in games:
                count+=1
                get_game_data(g, count, playoffs)
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)

In [ ]:
#PLAYOFFS
attempt = 0
for attempt in range(5):
    print("attempt: " + str(attempt))
    try:
        teams_found = db.teams.find()
        team_count = 0
        for team in teams_found:
            team_count+=1
            print()
            print(str(team_count), "From team: ", team["name"])
            time.sleep(sleeptime)
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="00", season_type_nullable="PlayIn")
            playins = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team["teamID"], league_id_nullable="00", season_type_nullable="Playoffs")
            games = gamefinder.get_normalized_dict()["LeagueGameFinderResults"]
            games = games + playins
            playoffs=True
            print(len(games), " total games")
            games = [game for game in games if int(game["SEASON_ID"][1:]) >= 2021]
            print(len(games), " recent games")
            found_games = [game for game in db.teamgames.find({"teamID":team["teamID"]})]
            print(len(found_games), " games found in db")
            games = [game for game in games if game_exists(found_games, game["GAME_ID"]) == False]
            print(len(games), " games not in db")

            count=0
            for g in games:
                count+=1
                get_game_data(g, count, playoffs)
        break
    except Exception as e:
        print("failed on attempt: " + str(attempt))
        print(e)
        # print(game)
        time.sleep(sleeptime*10)